In [1]:
import numpy as np
import pandas as pd

"""
model.py

Simulates basic STICS model outputs for a maize field based on user inputs.

Inputs:
    soil: dict containing soil properties
        - texture: str (e.g., 'Loamy')
        - depth: float (m)
        - field_capacity: float (m3/m3)
        - wilting_point: float (m3/m3)
        - initial_water: float (mm)
        - initial_nitrate: float (kg/ha)

    weather: pd.DataFrame with daily columns:
        - date: datetime
        - tmin: float (°C)
        - tmax: float (°C)
        - precipitation: float (mm)
        - radiation: float (MJ/m2)

    crop: dict with crop management settings
        - variety: str
        - sowing_date: datetime
        - density: float (plants/m2)
        - sowing_depth: float (cm)
        - fertilizer_schedule: list of tuples (date, amount_kgN/ha)
        - irrigation_schedule: list of tuples (date, amount_mm)

    parameters: dict of model parameters
        - light_use_efficiency: float (gDM/MJ)
        - max_leaf_area_index: float
        - root_depth_rate: float (cm/day)
        - n_uptake_efficiency: float (kgN/kgDM)

Outputs:
    results: dict of pd.DataFrame with time series:
        - lai: Leaf Area Index
        - biomass: Total aboveground biomass (g/m2)
        - yield: Grain yield (g/m2)
        - soil_water: Available soil water (mm)
        - soil_nitrate: Soil nitrate content (kg/ha)
        - water_balance: Precipitation, evapotranspiration, drainage
        - n_balance: N inputs, uptake, losses

Example usage:
    from model import run_simulation
    results = run_simulation(soil, weather, crop, parameters)
    print(results['lai'].head())
"""

def run_simulation(soil, weather, crop, parameters):
    # Prepare time axis
    df = weather.copy().reset_index(drop=True)
    n = len(df)

    # Initialize output arrays
    lai = np.zeros(n)
    biomass = np.zeros(n)
    soil_water = np.zeros(n)
    soil_nitrate = np.zeros(n)
    water_upt = np.zeros(n)
    drainage = np.zeros(n)
    n_uptake = np.zeros(n)

    # Initial states
    soil_water[0] = soil['initial_water']
    soil_nitrate[0] = soil['initial_nitrate']

    # Phenological tracking (simple degree-day)
    base_temp = 8.0  # °C
    cum_dd = 0.0
    dd_emergence = 100
    dd_maturity = 1200

    for i in range(1, n):
        # Degree-days
        tavg = (df.loc[i, 'tmax'] + df.loc[i, 'tmin']) / 2
        dd = max(0, tavg - base_temp)
        cum_dd += dd

        # LAI development
        if cum_dd < dd_emergence:
            lai[i] = 0.1
        elif cum_dd < dd_maturity:
            lai[i] = min(parameters['max_leaf_area_index'],
                        parameters['max_leaf_area_index'] * ((cum_dd - dd_emergence) / (dd_maturity - dd_emergence)))
        else:
            lai[i] = max(0, parameters['max_leaf_area_index'] * (1 - (cum_dd - dd_maturity) / (dd_maturity * 0.2)))

        # Radiation interception and biomass
        rad = df.loc[i, 'radiation']
        intercepted = rad * (1 - np.exp(-0.5 * lai[i]))
        dm_gain = intercepted * parameters['light_use_efficiency']
        biomass[i] = biomass[i-1] + dm_gain

        # Soil water balance
        precip = df.loc[i, 'precipitation']
        et0 = 0.5 * rad  # reference ET (simplified)
        et_crop = et0 * (lai[i] / parameters['max_leaf_area_index'])
        inflow = precip + sum(amount for date, amount in crop.get('irrigation_schedule', []) if date == df.loc[i, 'date'])
        outflow = et_crop
        soil_water[i] = max(0, soil_water[i-1] + inflow - outflow)
        drainage[i] = max(0, (soil_water[i] - soil['field_capacity']) if soil_water[i] > soil['field_capacity'] else 0)

        # Nitrogen uptake and balance
        # Fertilizer application
        fert = sum(amount for date, amount in crop.get('fertilizer_schedule', []) if date == df.loc[i, 'date'])
        soil_nitrate[i] = soil_nitrate[i-1] + fert
        # Uptake
        n_uptake[i] = dm_gain * parameters['n_uptake_efficiency']
        soil_nitrate[i] = max(0, soil_nitrate[i] - n_uptake[i])

    # Package results
    results = {
        'lai': pd.DataFrame({'date': df['date'], 'lai': lai}),
        'biomass': pd.DataFrame({'date': df['date'], 'biomass': biomass}),
        'yield': pd.DataFrame({'date': df['date'], 'yield': biomass * 0.45}),  # partition to grain
        'soil_water': pd.DataFrame({'date': df['date'], 'soil_water': soil_water}),
        'soil_nitrate': pd.DataFrame({'date': df['date'], 'soil_nitrate': soil_nitrate}),
        'water_balance': pd.DataFrame({'date': df['date'], 'precip': df['precipitation'],
                                       'et': et_crop, 'drainage': drainage}),
        'n_balance': pd.DataFrame({'date': df['date'], 'n_uptake': n_uptake, 'fert': [sum(amount for d, amount in crop.get('fertilizer_schedule', []) if d == date) for date in df['date']]})
    }

    return results

if __name__ == '__main__':
    # Example usage with dummy inputs
    from datetime import datetime, timedelta

    # Generate dummy weather for one season
    start = datetime(2024, 4, 18)
    dates = [start + timedelta(days=i) for i in range(150)]
    weather = pd.DataFrame({
        'date': dates,
        'tmin': np.random.uniform(10, 15, size=150),
        'tmax': np.random.uniform(20, 30, size=150),
        'precipitation': np.random.uniform(0, 10, size=150),
        'radiation': np.random.uniform(10, 25, size=150),
    })

    soil = {
        'texture': 'Loamy',
        'depth': 1.2,
        'field_capacity': 0.30,
        'wilting_point': 0.12,
        'initial_water': 150,
        'initial_nitrate': 50,
    }

    crop = {
        'variety': 'Maize Hybrid A',
        'sowing_date': start,
        'density': 9,
        'sowing_depth': 5,
        'fertilizer_schedule': [(start + timedelta(days=30), 100)],
        'irrigation_schedule': [(start + timedelta(days=45), 20)],
    }

    parameters = {
        'light_use_efficiency': 1.5,
        'max_leaf_area_index': 5.0,
        'root_depth_rate': 0.8,
        'n_uptake_efficiency': 0.03,
    }

    results = run_simulation(soil, weather, crop, parameters)
    print(results['lai'].head())


        date  lai
0 2024-04-18  0.0
1 2024-04-19  0.1
2 2024-04-20  0.1
3 2024-04-21  0.1
4 2024-04-22  0.1


In [5]:
import cdsapi
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime

# Initialize the CDS API client
c = cdsapi.Client()

# Mapping from ERA5-Land variables to our names
_VARIABLES = {
    '2m_temperature': 't2m',
    'total_precipitation': 'tp',
    'surface_net_solar_radiation': 'ssr'
}

# ERA5-Land expects dates in YYYY-MM-DD

def fetch_weather(lat, lon, start_date, end_date, output_path='/tmp/era5_land.nc'):
    """
    Downloads and processes daily weather data from Copernicus ERA5-Land for a point.

    Parameters:
    -----------
    lat, lon : float
        Geographic coordinates of the point (decimal degrees).
    start_date, end_date : str or datetime
        Start and end dates in 'YYYY-MM-DD' or datetime.
    output_path : str
        Path to save the downloaded NetCDF file.

    Returns:
    --------
    pandas.DataFrame with columns:
      - date: datetime
      - tmin: °C
      - tmax: °C
      - precipitation: mm/day
      - radiation: MJ/m2/day
    """
    # Format dates
    if isinstance(start_date, datetime):
        start = start_date.strftime('%Y-%m-%d')
    else:
        start = start_date
    if isinstance(end_date, datetime):
        end = end_date.strftime('%Y-%m-%d')
    else:
        end = end_date

    # Request ERA5-Land daily aggregated data
    c.retrieve(
        'reanalysis-era5-land',
        {
            'variable': list(_VARIABLES.keys()),
            'year': [d[:4] for d in pd.date_range(start, end, freq='D').strftime('%Y')],
            'month': [d[5:7] for d in pd.date_range(start, end, freq='D').strftime('%m')],
            'day': [d[8:10] for d in pd.date_range(start, end, freq='D').strftime('%d')],
            'time': ['00:00'],  # daily aggregated fields
            'format': 'netcdf',
            'area': [lat + 0.0005, lon - 0.0005, lat - 0.0005, lon + 0.0005],  # small box around point
        },
        output_path
    )

    # Load and extract point time series
    ds = xr.open_dataset(output_path)

    # Select nearest grid point
    ds_point = ds.sel(
        latitude=lat, longitude=lon, method='nearest'
    )

    # Convert units and aggregate
    df = ds_point.to_dataframe()
    df = df.reset_index()
    df['date'] = pd.to_datetime(df['time']).dt.date

    # Daily min/max temperature (K to °C)
    # ERA5-Land 't2m' is hourly; but here we requested daily, so treat as daily mean
    df_daily = df.groupby('date').agg({
        't2m': ['min', 'max', 'mean'],
        'tp': 'last',
        'ssr': 'last'
    })
    df_daily.columns = ['tmin_k', 'tmax_k', 'tmean_k', 'precipitation_m', 'radiation_j']

    # Convert to desired units
    df_daily['tmin'] = df_daily['tmin_k'] - 273.15
    df_daily['tmax'] = df_daily['tmax_k'] - 273.15
    # precipitation in m to mm
    df_daily['precipitation'] = df_daily['precipitation_m'] * 1000
    # radiation in J/m2 to MJ/m2
    df_daily['radiation'] = df_daily['radiation_j'] / 1e6

    # Final DataFrame
    weather = df_daily[['tmin', 'tmax', 'precipitation', 'radiation']].reset_index()
    return weather

# Example usage
def __main__():
    lat, lon = 45.78, 3.08  # Clermont-Ferrand
    start = '2025-04-20'
    end = '2025-09-15'
    df = fetch_weather(lat, lon, start, end)
    print(df.head())

if __name__ == '__main__':
    __main__()


2025-05-05 19:28:44,871 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-05 19:28:44,872 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


HTTPError: 403 Client Error: Forbidden for url: https://cds.climate.copernicus.eu/api/retrieve/v1/processes/reanalysis-era5-land/execution
cost limits exceeded
Your request is too large, please reduce your selection.

In [8]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta


def fetch_historical(lat, lon, start_date, end_date, timezone="Europe/Paris"):
    """
    Fetch past weather data using the Open-Meteo Archive API.
    """
    url = (
        "https://archive-api.open-meteo.com/v1/archive"
        f"?latitude={lat}&longitude={lon}"
        f"&start_date={start_date}&end_date={end_date}"
        "&daily=temperature_2m_min,temperature_2m_max,precipitation_sum,shortwave_radiation_sum"
        f"&timezone={timezone}"
    )
    r = requests.get(url)
    r.raise_for_status()
    data = r.json().get("daily", {})

    radiation_raw = data.get("shortwave_radiation_sum", [])
    radiation = [(x * 0.0036) if x is not None else np.nan for x in radiation_raw]

    df = pd.DataFrame({
        "date": pd.to_datetime(data.get("time", [])),
        "tmin": data.get("temperature_2m_min", []),
        "tmax": data.get("temperature_2m_max", []),
        "precipitation": data.get("precipitation_sum", []),
        "radiation": radiation,
    })
    return df


def fetch_moving_avg_forecast(lat, lon, start_date, end_date, years=4):
    """
    Estimate future weather by averaging historical data for the same calendar days
    over the previous `years` years.

    Returns a DataFrame covering [start_date..end_date], with columns: date, tmin, tmax,
    precipitation, radiation.
    """
    sd = pd.to_datetime(start_date).date()
    ed = pd.to_datetime(end_date).date()
    frames = []

    # Collect historical series for each of the previous `years` years
    for y in range(1, years+1):
        year_offset_sd = sd.replace(year=sd.year - y)
        year_offset_ed = ed.replace(year=ed.year - y)
        df_hist = fetch_historical(lat, lon,
                                   year_offset_sd.isoformat(),
                                   year_offset_ed.isoformat())
        # Align dates to forecast year
        df_hist['day_of_year'] = df_hist['date'].dt.strftime('%m-%d')
        frames.append(df_hist)

    # Concatenate and compute mean for each calendar day
    df_concat = pd.concat(frames, ignore_index=True)
    df_avg = df_concat.groupby('day_of_year').agg({
        'tmin': 'mean',
        'tmax': 'mean',
        'precipitation': 'mean',
        'radiation': 'mean'
    }).reset_index()
    # Reconstruct forecast dates
    forecast_days = (ed - sd).days + 1
    dates = [sd + timedelta(days=i) for i in range(forecast_days)]
    doy = [d.strftime('%m-%d') for d in dates]
    df_forecast = pd.DataFrame({
        'date': dates,
        'day_of_year': doy
    }).merge(df_avg, on='day_of_year', how='left')
    return df_forecast[['date', 'tmin', 'tmax', 'precipitation', 'radiation']]


def fetch_weather(lat, lon, start_date, end_date):
    """
    Unified function: fetch historical up to today-1, then estimate future via moving average.

    Returns a DataFrame with columns: date, tmin, tmax, precipitation, radiation.
    """
    sd = pd.to_datetime(start_date).date()
    ed = pd.to_datetime(end_date).date()
    today = date.today()
    yesterday = today - timedelta(days=1)

    parts = []
    # Historical part
    if sd <= yesterday:
        hist_end = min(ed, yesterday)
        parts.append(fetch_historical(lat, lon, sd.isoformat(), hist_end.isoformat()))

    # Future estimate part
    if ed >= today:
        future_start = max(sd, today)
        parts.append(fetch_moving_avg_forecast(lat, lon,
                                               future_start.isoformat(),
                                               ed.isoformat()))

    if not parts:
        raise ValueError("Requested range yields no data (all dates are in the future?).")

    df = pd.concat(parts).reset_index(drop=True)
    return df


# Tests
def _test_synthetic():
    # Synthetic historical stub for deterministic test
    def stub_hist(lat, lon, start, end):
        dates = pd.date_range(start, end)
        return pd.DataFrame({
            'date': dates,
            'tmin': np.arange(len(dates)),
            'tmax': np.arange(len(dates)) + 10,
            'precipitation': np.ones(len(dates)),
            'radiation': np.ones(len(dates)) * 5
        })

    global fetch_historical
    real_hist = fetch_historical
    fetch_historical = stub_hist

    # Test forecast for 3-day window, averaging stub years
    today = date.today()
    sd = (today + timedelta(days=1)).isoformat()
    ed = (today + timedelta(days=3)).isoformat()
    df = fetch_weather(0, 0, sd, ed)
    assert len(df) == 3 + 0  # no historical
    # tmin average of 0..2 across 4 years => same pattern
    print("Synthetic forecast test passed", df)

    fetch_historical = real_hist

if __name__ == "__main__":
    _test_synthetic()


Synthetic forecast test passed          date  tmin  tmax  precipitation  radiation
0  2025-05-06   0.0  10.0            1.0        5.0
1  2025-05-07   1.0  11.0            1.0        5.0
2  2025-05-08   2.0  12.0            1.0        5.0
